In [ ]:
import datetime
import json
from typing import Callable, List, Optional

# Basic Example

In [ ]:
# some simple function printing a string
def show_me(text: str):
    print(text, end="")
    
show_me("blue")

In [ ]:
# a function that takes another function and returns an altered function
# this will be used as our decorator
def sparkly(func: Callable) -> Callable:
    def inner(*args, **kwargs):
        print("~*~*", end="")
        func(*args, **kwargs)
        print("*~*~")
    return inner

In [ ]:
# using our one function as a decorator for the other
@sparkly
def show_me(text: str):
    print(text, end="")
    
show_me("blue")

In [ ]:
# this gives the same result as above, this is practically what the "decorating" does
def show_me(text: str):
    print(text, end="")
    
show_me = sparkly(show_me)
show_me("blue")

# Useful Examples

## Silencing a lot of output

Note: **don't** broadly silence errors

In [ ]:
!pip install loguru

In [ ]:
import os
import sys
from contextlib import redirect_stdout

from loguru import logger
logger.remove()  # removing the logging handler initialized by jupyter

In [ ]:
# lets pretend this is a function we have no direct control over but we import and use it
def noisy_imported_function() -> int:
    logger.add(sys.stdout, level="INFO")
    logger.info("we will start now!")
    a = 2 + 2
    logger.success("that went well")
    b = 1 + 3
    logger.success("nice going")
    if a == b:
        logger.info("we entered the if")
        c = 2 * a
    else:
        logger.warning("an if went to the else!")
        c = a + b
    logger.info("the func is done and will return now")
    logger.remove()
    return c

In [ ]:
noisy_imported_function()

In [ ]:
# our decorator
def silent(func: Callable) -> Callable:
    def inner(*args, **kwargs):
        with redirect_stdout(os.devnull):
            func(*args, **kwargs)
    return inner

In [ ]:
@silent
def my_usecase():
    # doing something
    c = noisy_imported_function()
    return c + 1

my_usecase()
print("enjoy the silence")

## Adding a timer to a function

In [ ]:
import time

In [ ]:
def timed(func: Callable) -> Callable:
    def inner(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print(f"took {end - start:.2f} seconds")
    return inner

In [ ]:
@timed
def expensive_function():
    for i in range(5):
        print(".", end="")
        time.sleep(1)
    print("done")
    
expensive_function()

# Built-in Decorators

## @classmethod and @staticmethod

Note: Based on mCoding's [Python staticmethod and classmethod video](https://www.youtube.com/watch?v=SXApHXsDe8I)

In [ ]:
class Calendar:
    def __init__(self, events: Optional[List[str]] = None):
        if events is None:
            events = []
        self.events = events
    
    def add_event(self, event: str):
        self.events.append(event)
    
    # @staticmethod
    def is_weekend(self, date: datetime.date) -> bool:
        return date.weekday() > 4


In [ ]:
my_calendar = Calendar()
my_calendar.add_event("Python Learning Group")
my_calendar.events

In [ ]:
todays_date = datetime.datetime.now().date()

In [ ]:
my_calendar.is_weekend(todays_date)

In [ ]:
Calendar().is_weekend(todays_date)

In [ ]:
Calendar.is_weekend(todays_date)

Try making `is_weekend` a `staticmethod`.

In [ ]:
class Calendar:
    def __init__(self, events: Optional[List[str]] = None):
        if events is None:
            events = []
        self.events = events
    
    def add_event(self, event: str):
        self.events.append(event)

    # @classmethod
    def from_txt(self, path: str):
        """Alternative constructor"""
        with open(path) as file_ref:
            return Calendar(file_ref.read().splitlines())

In [ ]:
my_calendar = Calendar()
new_calendar = my_calendar.from_txt("exported_events.txt")
new_calendar.events

In [ ]:
your_calendar = Calendar.from_txt("exported_events.txt")
your_calendar.events

Try making `from_txt` a `classmethod`.

TLDR:
* **instance** method: implicit first `self` parameter
* **static** method: no implict first paramter
* **class** method: implicit first `cls` parameter

Note: `self`, `cls` are convetions

In [ ]:
class StaticMethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        """
        Ignores instance and class it was called from
        """
        return self.func
    
    def __call_(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [ ]:
class ClassMethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        """
        Binds function to owner when getting function
        instead of instance
        """
        return self.func.__get__(owner)
    
    def __call_(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [ ]:
class Calendar:
    def __init__(self, events: Optional[List[str]] = None):
        if events is None:
            events = []
        self.events = events
    
    def add_event(self, event: str):
        self.events.append(event)

    @classmethod
    def from_txt(cls, path: str):
        """Alternative constructor"""
        with open(path) as file_ref:
            return cls(file_ref.read().splitlines())

class SpecialCalendar(Calendar):
    pass

In [ ]:
type(SpecialCalendar.from_txt("exported_events.txt"))

## @property

In [ ]:
class Person(object):
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"

In [ ]:
person = Person("Jeffrey", "Bezos")

In [ ]:
person.full_name()

method is now a attribute

In [ ]:
person.full_name

method is now a **read-only** attribute

In [ ]:
person.full_name = "Jeff Bezos"

changing composite updates the property

In [ ]:
person.first_name = "Jeff"
person.full_name

## @functools.cached_property

In [ ]:
!pip install numpy

In [ ]:
import statistics
from functools import cached_property

import numpy as np

In [ ]:
class DataSet:
    def __init__(self, numbers: List):
        self.data = numbers
        
    @property
    def stdev(self):
        return statistics.stdev(self.data)
    
    @cached_property
    def variance(self):
        return statistics.variance(self.data)

In [ ]:
measurements = DataSet(np.random.rand(1_000_000).tolist())

calculating normal property

In [ ]:
%%timeit -n 1 -r 1
measurements.stdev

reusing cached property

In [ ]:
%%timeit -n 1 -r 1
measurements.variance

In [ ]:
%%timeit -n 1 -r 1
measurements.variance

cached property is also overwritable

In [ ]:
measurements.variance = 0.

# Other Applications

## `pytest` Decorators

https://docs.pytest.org

In [ ]:
!pip install pytest

In [ ]:
from pathlib import Path

import pytest

### parametrize

In [ ]:
@pytest.mark.parametrize("x", [0, 1])
@pytest.mark.parametrize("y", [2, 3])
def test_foo(x, y):
    assert x != y

### fixture

In [ ]:
@pytest.fixture(scope="session")
def working_directory():
    return Path.cwd()

In [ ]:
def test_cwd(working_directory):
    assert working_directory.exists()

## CLI with `click`

https://click.palletsprojects.com

In [ ]:
!pip install click

In [ ]:
import click

In [ ]:
@click.command()
@click.option('--count', default=1, help='Number of greetings.')
@click.option('--name', prompt='Your name',
              help='The person to greet.')
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo(f"Hello {name}!")